In [1]:
# set your API Key in .env
%load_ext dotenv
%dotenv

import os
from pprint import pprint
from girder_client import GirderClient
from dive_utils import fromMeta

GIRDER_API_KEY=os.getenv('GIRDER_API_KEY', 'CHANGEME')
diveclient = GirderClient(apiUrl='https://viame.kitware.com/api/v1')
token = diveclient.authenticate(apiKey=GIRDER_API_KEY)
print(f'Authenticated!')

Authenticated!


In [2]:
from typing import Callable, Iterable
from dive_utils.constants import PublishedMarker

def unpaginate_documents(getter: Callable[[int, int], Iterable]):
    """
    Tages a paginated getter and unravels it into an array of items
    """
    limit = 1000
    offset = 0
    documents = []
    page = []

    while True:
        page = getter(limit, offset)
        documents.extend(page)
        offset += limit
        if len(page) == 0:
            break;

    return documents

def getDatasets(limit, offset):
    return diveclient.get(
        'viame/datasets',
        parameters={
            'limit': limit,
            'offset': offset,
        },
    )

def getUsers(limit, offset):
    return diveclient.get(
        'user',
        parameters={
            'limit': limit,
            'offset': offset,
        },
    )

def getJobs(limit, offset):
    return diveclient.get(
        'job',
        parameters={
            'limit': limit,
            'offset': offset,
        }
    )

def getSaves(limit, offset):
    return [item for item in diveclient.get(
        'item',
        parameters={
            'limit': limit,
            'offset': offset,
            'text': '*.json', 
        }
    ) if item['meta'].get('detection')]

def hostedData(limit, offset):
    return diveclient.get(
        'assetstore/5e4c1de1a0fc86aa03120c33/files',
        parameters={
            'limit': limit,
            'offset': offset,
        }
    )

In [7]:
users = unpaginate_documents(getUsers)

In [12]:
datasets = unpaginate_documents(getDatasets)

In [49]:
jobs = unpaginate_documents(getJobs)

In [69]:
saves = unpaginate_documents(getSaves)

In [78]:
items = unpaginate_documents(hostedData)

In [9]:
%matplotlib widget
import datetime
import matplotlib.pyplot as plt

def parseTime(s):
    try:
        return datetime.datetime.strptime(s, '%Y-%m-%dT%H:%M:%S.%f+00:00')
    except ValueError:
        return datetime.datetime.strptime(s, '%Y-%m-%dT%H:%M:%S+00:00')

def makePlot(items, xkey, ykey, label):

    x = sorted([parseTime(item['created']) for item in items])
    y = []
    last = 0
    for i, item in enumerate(items):
        if ykey:
            y.append(last + item[ykey])
            last = last + item[ykey]
        else:
            y.append(i + 1)
            last = i + 1

    fig, ax = plt.subplots()
    ax.set(xlabel='Time', ylabel=label)
    ax.plot(x, y)
    plt.gcf().autofmt_xdate()
    plt.show()

In [13]:
makePlot(datasets,"created", None, "New datasets")
# makePlot(jobs,"created", "Job executions")
makePlot(users, "created", None, "New registrations")
# makePlot(saves,"created", "Save events")
# makePlot(items,"created", "Hosted data (Terrabytes)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …